In [0]:
import  numpy as  np
import pandas as pd
import os 
import sys
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.models import model_from_json
from keras.preprocessing import image
from keras.layers import  Layer
from keras.preprocessing.image import ImageDataGenerator


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!cp drive/'My Drive'/fer2013.csv .
df=pd.read_csv('fer2013.csv')
print(df.info())
print(df["Usage"].value_counts())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [36]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index,row in df.iterrows():
  val = row['pixels'].split(" ")
  try:
    if 'Training' in row['Usage']:
      X_train.append(np.array(val,'float32'))
      train_y.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
      X_test.append(np.array(val,'float32'))
      test_y.append(row['emotion'])
  except:
    print(f"error occured at index:{index} and row:{row}")
  
# print(f"X_train sample data:{X_train[0:2]}")
# print(f"train_y sample data:{train_y[0:2]}")
# print(f"X_test sample data:{X_test[0:2]}")
# print(f"test_y sample data:{test_y[0:2]}")

X_train=np.array(X_train,'float32')
train_y=np.array(train_y,'float32')
X_test=np.array(X_test,'float32')
test_y=np.array(test_y,'float32')


[[ 70.  80.  82. ... 106. 109.  82.]
 [151. 150. 147. ... 193. 183. 184.]
 [231. 212. 156. ...  88. 110. 152.]
 ...
 [ 74.  81.  87. ... 188. 187. 187.]
 [222. 227. 203. ... 136. 136. 134.]
 [195. 199. 205. ...   6.  15.  38.]]


(28709, 2304)

In [0]:
# Normalising databetween 0 and 1
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width,height = 48,48

train_y = np_utils.to_categorical(train_y,num_classes=48)
test_y = np_utils.to_categorical(test_y,num_classes=48)


X_train-=np.mean(X_train,axis=0)
X_train/=np.std(X_train,axis=0)

X_test-=np.mean(X_test,axis=0)
X_test/=np.std(X_test,axis=0)

X_train = X_train.reshape(X_train.shape[0],width,height,1)
X_test = X_test.reshape(X_test.shape[0],width,height,1)




In [40]:
#cnn Model

model = Sequential()
# first CNN layer
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu',input_shape=(X_train.shape[1:])))
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

# Second CNN Layer
# model.add(Conv2D(2*num_features,(3,3), activation='relu'))
# model.add(Conv2D((2*num_features,(3,3),activation='relu'))

# model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
# model.add(Dropout(0.5))

# Third CNN Layer
model.add(Conv2D(2*2*num_features,(3,3),activation='relu'))
model.add(Conv2D(2*3*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels,activation='softmax'))

# model.summery()
gen = ImageDataGenerator()
train_generator = gen.flow(X_train, train_y, batch_size=batch_size)

model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

model.fit(X_train,train_y,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,test_y),shuffle=True)

# save model
fer_json = model.to_json()
with open("fer.json","w") as json_file:
  json_file.write(fer_json)

model.save_weights("fer.h5")

ValueError: ignored